In [45]:
def get_db_url():
    from env import host, user, password
    
    return "url = f'mysql+pymysql://{user}:{password}@{host}/employees'"

In [46]:
get_db_url()

ImportError: cannot import name 'user' from 'env' (/Users/brandonnavarrete/codeup-data-science/data-science-libraries/env.py)